# Setting up the Knowledge Graph Datasets

In [1]:
import sys
sys.path.append('..')
from aips import *
import os
from IPython.core.display import display,HTML
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col
spark = SparkSession.builder.appName("ch7").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Starting the Reviews Search Web Server and Launching the Search Page

### Listing 7.2

In [2]:
def get_running_webservers():
    already_running_webservers = ! ps -ef | grep '[s]tart-webserver.py' | awk '{print $2}'
    return already_running_webservers
    
def stop_running_webservers():
    already_running_webservers = get_running_webservers()
    for pid in already_running_webservers:
        print("Stopping webserver (pid: " + pid + ")")
        results = ! xargs kill -9 {pid}

def start_reviews_search_webserver():
    stop_running_webservers() #in case it was already running
    ! pip install staticmap
    get_ipython().system = os.system
    ! cd ../webserver && python start-webserver.py &
    if len(get_running_webservers()) > 0:
        print("Successfully Started Webserver (pid: " + get_running_webservers()[0] + ")!")

In [3]:
#Start the web server
start_reviews_search_webserver()

Stopping webserver (pid: 3068)
Successfully Started Webserver (pid: 4706)!


In [4]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [5]:
from IPython.display import IFrame
src = f'{WEBSERVER_URL}/search?q=bbq'
IFrame(src=src, width="100%", height=800)

172.31.0.1 - - [05/Oct/2022 16:06:59] "GET /search?q=bbq HTTP/1.1" 200 -
172.31.0.1 - - [05/Oct/2022 16:06:59] "GET /search?q=bbq HTTP/1.1" 200 -
172.31.0.1 - - [05/Oct/2022 16:07:05] "POST /process_basic_query HTTP/1.1" 200 -
172.31.0.1 - - [05/Oct/2022 16:07:08] "POST /process_basic_query HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('172.31.0.1', 56330)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.9/http/server.py", line 653, in __init__
    super().__init__(*args, **kwargs)
  File "/opt/conda/lib/py

In [6]:
%%html
<iframe src="http://localhost:2345/search?q=bbq" width=100% height="800"></iframe>


172.31.0.1 - - [05/Oct/2022 16:07:22] "POST /process_basic_query HTTP/1.1" 200 -


In [7]:
#Cleanup so webserver doesn't keep running after you're done
stop_running_webservers()

Stopping webserver (pid: 4706)


## Success!
